# Importing Libraries

In [73]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer 

#nltk.download('all')

# Reading Docs

In [74]:
path = "Docs/"

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        files.append(os.path.join(r, file))
    
for f in files:
    print(f)

Docs/file1
Docs/file2
Docs/file3.txt


# Removing Headers and HTML Tags, Tokenization, Lower Casing, StopListing, Stemming

In [75]:
docs_words = {}
i = 0 #for docs_words

f = open("stoplist.txt", "r") 
stoplist = f.read().splitlines() #Stoplist words

ps = PorterStemmer() #stemmer

for f in files:
    doc = open(f,"r")
    html = doc.read()
    
    index = html.find("<html")
    if index == -1 :
        html.find("<Html")
    elif index == -1 : 
        html.find("<HTML")
    html = html[index:]

    soup = BeautifulSoup(html)

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    
    #print("Doc : " + text + "\n\n")
    tknzr = TweetTokenizer()
    tokens = tknzr.tokenize(text)
    words_pos = {}
    j = 0; #for words_pos
    for t in tokens:
        t = t.lower() #lowering Case
        t = ps.stem(t) #stemming
        if t not in stoplist and len(t) > 1: #Removing stop words
            if t not in words_pos.keys():
                words_pos[t] = []
            if t in words_pos.keys():
                words_pos[t].append(j)
            j = j + 1
    
    docs_words[i] = words_pos
    i = i + 1

print(docs_words) 

{0: {'dark': [0, 18, 27, 54, 69, 96, 104, 134], "chocolate'": [1, 55, 187, 198, 218], 'health': [2, 5, 23, 56, 188, 200, 235], 'benefit': [3, 189], 'chocol': [4, 11, 16, 19, 28, 38, 70, 97, 105, 124, 135, 158, 163, 167, 172, 177, 214, 221, 232, 234], 'basic': [6, 7, 202], "it'": [8], 'wish': [9], 'think': [10], 'good': [12, 99, 175], 'studi': [13, 51, 84, 95, 201], 'show': [14, 113], 'eat': [15], 'primarili': [17], 'contribut': [20], 'improv': [21, 110, 194], 'cardiovascular': [22, 87], 'pack': [24], 'natur': [25, 48, 75, 159], 'antioxid': [26, 66, 67, 74, 86, 203], 'cocoa': [29, 42, 62, 71, 98], 'sit': [30], 'good-for-y': [31], 'categori': [32], 'green': [33], 'tea': [34], 'blueberri': [35], "that'": [36], 'becaus': [37, 58], 'come': [39], 'cacao': [40, 45, 227], 'bean': [41, 43], 'grow': [44], 'tree': [46], 'full': [47], 'plant': [49], 'nutrient': [50, 166], 'date': [52], 'highlight': [53], 'valu': [57], 'ha': [59, 106, 136, 254, 260], 'highest': [60], 'percentag': [61], 'solid': [63